In [99]:
import requests
import csv
from datetime import datetime, timedelta

In [100]:
API_KEY = 'insert api'
BASE_URL = 'https://aeroapi.flightaware.com/aeroapi'

headers = {
    'x-apikey': API_KEY
}

In [101]:
def get_airport_info(airport_code):
    endpoint = f'{BASE_URL}/airports/{airport_code}'
    response = requests.get(endpoint, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        return response.text

airport_code = 'WMKK'
airport_info = get_airport_info(airport_code)
print(airport_info)


{'airport_code': 'WMKK', 'alternate_ident': 'KUL', 'code_icao': 'WMKK', 'code_iata': 'KUL', 'code_lid': None, 'name': "Kuala Lumpur Int'l", 'type': 'Airport', 'elevation': 70, 'city': 'Sepang', 'state': 'Selangor', 'longitude': 101.698056, 'latitude': 2.743333, 'timezone': 'Asia/Kuala_Lumpur', 'country_code': 'MY', 'wiki_url': 'https://en.wikipedia.org/wiki/Kuala_Lumpur_International_Airport', 'airport_flights_url': '/airports/WMKK/flights', 'alternatives': []}


# Arrival

In [112]:
import requests
import csv
from datetime import datetime, timedelta

API_KEY = 'UiuE73uNUuG9AwnT1pRMOLayYw6ZjX4z'
BASE_URL = 'https://aeroapi.flightaware.com/aeroapi'

headers = {
    'x-apikey': API_KEY
}

def convert_myt_to_utc(myt_time_str):
    # Parse MYT time string
    myt_time = datetime.strptime(myt_time_str, '%Y-%m-%dT%H:%M:%S')
    # Subtract 8 hours to convert MYT to UTC
    utc_time = myt_time - timedelta(hours=8)
    return utc_time.isoformat()

def get_recent_arrivals(airport_code, start_time_myt):
    
    # Convert MYT times to UTC ISO 8601 format
    end_time_iso = convert_myt_to_utc(end_time_myt) + 'Z'
    start_time_iso = convert_myt_to_utc(start_time_myt) + 'Z'
    
    # Endpoint for arrivals
    endpoint = f'{BASE_URL}/airports/{airport_code}/flights/arrivals'
    params = {
        'start': start_time_iso,
        'end': end_time_iso
    }
    
    response = requests.get(endpoint, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return response.text

def sanitize_filename(filename):
    # Replace problematic characters with underscores
    return filename.replace(':', '_')

def save_to_csv(data, start_time_myt):
    # Generate and sanitize filename based on start time in MYT
    filename = f'arrivals_{sanitize_filename(start_time_myt)}.csv'
    
    # Open file in write mode
    with open(filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        # Write header
        writer.writerow(['Flight ID', 'Flight Number', 'Aircraft Type', 'Scheduled Arrival', 'Actual Arrival', 'Origin', 'Destination', 'Gate Destination'])
        
        # Write flight details
        for flight in data.get('arrivals', []):
            writer.writerow([
                flight.get('ident', 'N/A'),
                flight.get('flight_number', 'N/A'),
                flight.get('aircraft_type', 'N/A'),
                flight.get('scheduled_on', 'N/A'),
                flight.get('actual_on', 'N/A'),
                flight.get('origin', {}).get('name', 'N/A'),
                flight.get('destination', {}).get('name', 'N/A'),
                flight.get('gate_destination', 'N/A')
            ])

# Malaysia Time for the start of the time range
start_time_myt = '2024-08-27T10:00:00'
end_time_myt = '2024-08-27T11:00:00'


# Fetch recent arrivals
airport_code = 'WMKK'
recent_arrivals = get_recent_arrivals(airport_code, start_time_myt)

# Check if the data is in JSON format
if isinstance(recent_arrivals, dict):
    save_to_csv(recent_arrivals, start_time_myt)
    print("done")
else:
    print("Error fetching data:", recent_arrivals)

done


# Departure

In [113]:

def convert_myt_to_utc(myt_time_str):
    # Parse MYT time string
    myt_time = datetime.strptime(myt_time_str, '%Y-%m-%dT%H:%M:%S')
    # Subtract 8 hours to convert MYT to UTC
    utc_time = myt_time - timedelta(hours=8)
    return utc_time.isoformat()

def get_recent_departures(airport_code, start_time_myt):
    # Malaysia Time for the time range
    end_time_myt = '2024-08-27T11:00:00'
    
    # Convert MYT times to UTC ISO 8601 format
    end_time_iso = convert_myt_to_utc(end_time_myt) + 'Z'
    start_time_iso = convert_myt_to_utc(start_time_myt) + 'Z'
    
    # Endpoint for departures
    endpoint = f'{BASE_URL}/airports/{airport_code}/flights/departures'
    params = {
        'start': start_time_iso,
        'end': end_time_iso
    }
    
    response = requests.get(endpoint, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return response.text

def sanitize_filename(filename):
    # Replace problematic characters with underscores
    return filename.replace(':', '_')

def save_to_csv(data, start_time_myt):
    # Generate and sanitize filename based on start time in MYT
    filename = f'departures_{sanitize_filename(start_time_myt)}.csv'
    
    # Open file in write mode
    with open(filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        # Write header
        writer.writerow(['Flight ID', 'Flight Number', 'Aircraft Type', 'Scheduled Departure', 'Actual Departure', 'Origin', 'Destination', 'Gate Destination'])
        
        # Write flight details
        for flight in data.get('departures', []):
            writer.writerow([
                flight.get('ident', 'N/A'),
                flight.get('flight_number', 'N/A'),
                flight.get('aircraft_type', 'N/A'),
                flight.get('scheduled_off', 'N/A'),
                flight.get('actual_off', 'N/A'),
                flight.get('origin', {}).get('name', 'N/A'),
                flight.get('destination', {}).get('name', 'N/A'),
                flight.get('gate_destination', 'N/A')
            ])

# Malaysia Time for the start of the time range
start_time_myt = '2024-08-27T10:00:00'

# Fetch recent departures
airport_code = 'WMKK'
recent_departures = get_recent_departures(airport_code, start_time_myt)

# Check if the data is in JSON format
if isinstance(recent_departures, dict):
    save_to_csv(recent_departures, start_time_myt)
    print("done")
else:
    print("Error fetching data:", recent_departures)

done


# Arrival in MYT

In [114]:

def convert_myt_to_utc(myt_time_str):
    # Parse MYT time string
    myt_time = datetime.strptime(myt_time_str, '%Y-%m-%dT%H:%M:%S')
    # Subtract 8 hours to convert MYT to UTC
    utc_time = myt_time - timedelta(hours=8)
    return utc_time.isoformat()

def convert_utc_to_myt(utc_time_str):
    # Parse UTC time string
    utc_time = datetime.strptime(utc_time_str, '%Y-%m-%dT%H:%M:%SZ')
    # Add 8 hours to convert UTC to MYT
    myt_time = utc_time + timedelta(hours=8)
    return myt_time.strftime('%Y-%m-%d %H:%M:%S')

def get_recent_arrivals(airport_code, start_time_myt, end_time_myt):
    # Convert MYT times to UTC ISO 8601 format
    end_time_iso = convert_myt_to_utc(end_time_myt) + 'Z'
    start_time_iso = convert_myt_to_utc(start_time_myt) + 'Z'
    
    # Endpoint for arrivals
    endpoint = f'{BASE_URL}/airports/{airport_code}/flights/arrivals'
    params = {
        'start': start_time_iso,
        'end': end_time_iso
    }
    
    response = requests.get(endpoint, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return response.text

def sanitize_filename(filename):
    # Replace problematic characters with underscores
    return filename.replace(':', '_')

def save_to_csv(data, start_time_myt):
    # Generate and sanitize filename based on start time in MYT
    filename = f'arrivals_{sanitize_filename(start_time_myt)}.csv'
    
    # Open file in write mode
    with open(filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        # Write header
        writer.writerow(['Flight ID', 'Flight Number', 'Aircraft Type', 'Scheduled Arrival (MYT)', 'Actual Arrival (MYT)', 'Origin', 'Destination', 'Gate Destination'])
        
        # Write flight details
        for flight in data.get('arrivals', []):
            writer.writerow([
                flight.get('ident', 'N/A'),
                flight.get('flight_number', 'N/A'),
                flight.get('aircraft_type', 'N/A'),
                convert_utc_to_myt(flight.get('scheduled_on', 'N/A')),
                convert_utc_to_myt(flight.get('actual_on', 'N/A')),
                flight.get('origin', {}).get('name', 'N/A'),
                flight.get('destination', {}).get('name', 'N/A'),
                flight.get('gate_destination', 'N/A')
            ])

# Malaysia Time for the start and end of the time range
start_time_myt = '2024-08-27T10:00:00'
end_time_myt = '2024-08-27T11:00:00'

# Fetch recent arrivals
airport_code = 'WMKK'
recent_arrivals = get_recent_arrivals(airport_code, start_time_myt, end_time_myt)

# Check if the data is in JSON format
if isinstance(recent_arrivals, dict):
    save_to_csv(recent_arrivals, start_time_myt)
    print("done")
else:
    print("Error fetching data:", recent_arrivals)

done


# Departure in MYT

In [115]:
import requests
import csv
from datetime import datetime, timedelta

API_KEY = 'UiuE73uNUuG9AwnT1pRMOLayYw6ZjX4z'
BASE_URL = 'https://aeroapi.flightaware.com/aeroapi'

headers = {
    'x-apikey': API_KEY
}

def convert_myt_to_utc(myt_time_str):
    # Parse MYT time string
    myt_time = datetime.strptime(myt_time_str, '%Y-%m-%dT%H:%M:%S')
    # Subtract 8 hours to convert MYT to UTC
    utc_time = myt_time - timedelta(hours=8)
    return utc_time.isoformat()

def convert_utc_to_myt(utc_time_str):
    # Parse UTC time string
    utc_time = datetime.strptime(utc_time_str, '%Y-%m-%dT%H:%M:%SZ')
    # Add 8 hours to convert UTC to MYT
    myt_time = utc_time + timedelta(hours=8)
    return myt_time.strftime('%Y-%m-%d %H:%M:%S')

def get_recent_departures(airport_code, start_time_myt):
    # Malaysia Time for the time range
    end_time_myt = '2024-08-27T11:00:00'
    
    # Convert MYT times to UTC ISO 8601 format
    end_time_iso = convert_myt_to_utc(end_time_myt) + 'Z'
    start_time_iso = convert_myt_to_utc(start_time_myt) + 'Z'
    
    # Endpoint for departures
    endpoint = f'{BASE_URL}/airports/{airport_code}/flights/departures'
    params = {
        'start': start_time_iso,
        'end': end_time_iso
    }
    
    response = requests.get(endpoint, headers=headers, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        return response.text

def sanitize_filename(filename):
    # Replace problematic characters with underscores
    return filename.replace(':', '_')

def save_to_csv(data, start_time_myt):
    # Generate and sanitize filename based on start time in MYT
    filename = f'departures_{sanitize_filename(start_time_myt)}.csv'
    
    # Open file in write mode
    with open(filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        # Write header
        writer.writerow(['Flight ID', 'Flight Number', 'Aircraft Type', 'Scheduled Departure (MYT)', 'Actual Departure (MYT)', 'Origin', 'Destination', 'Gate Destination'])
        
        # Write flight details
        for flight in data.get('departures', []):
            writer.writerow([
                flight.get('ident', 'N/A'),
                flight.get('flight_number', 'N/A'),
                flight.get('aircraft_type', 'N/A'),
                convert_utc_to_myt(flight.get('scheduled_off', 'N/A')),
                convert_utc_to_myt(flight.get('actual_off', 'N/A')),
                flight.get('origin', {}).get('name', 'N/A'),
                flight.get('destination', {}).get('name', 'N/A'),
                flight.get('gate_destination', 'N/A')
            ])

# Malaysia Time for the start of the time range
start_time_myt = '2024-08-27T10:00:00'

# Fetch recent departures
airport_code = 'WMKK'
recent_departures = get_recent_departures(airport_code, start_time_myt)

# Check if the data is in JSON format
if isinstance(recent_departures, dict):
    save_to_csv(recent_departures, start_time_myt)
    print("done")
else:
    print("Error fetching data:", recent_departures)


done
